<h1 style='text-align: center;'>
    Yaml Enhancements 
</h1>
<h3 style='opacity: 0.8; text-align: center;'> Enhanced syntax for yaml configuration files </h3>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sources import Cfg

### References
The value of any object define in the configuration can be "copied" using absolute reference:
`` field: $absolute.path``

or relative reference:
`` field: $.relative.path``

(Similarly to python relative import syntax, '``.``' allows to go up in the hierarchy.)

In [3]:
cfg = Cfg.Parser('yaml-demos/references.yaml').get_config(parse_obj=False)

with open('yaml-demos/references.yaml', 'r') as f:
    raw = f.read()
print(' === RAW FILE ===')
print(raw)
print('\n === PARSED CFG ===')
print(cfg.to_yaml())

assert cfg['references.absolute-ref1'] == 'foo value'
assert cfg['references.absolute-ref2.elem1'] == 'FOO 1'
assert cfg['references.absolute-ref2.elem2'] == 'FOO 2'
assert cfg['references.relative-ref1'] == 'foo value'
assert cfg['references.relative-ref2'] == 'FOO 2'

 === RAW FILE ===
references:
    absolute-ref1: $foo
    absolute-ref2: $interesting.stuff
    relative-ref1: $..foo
    relative-ref2: $.absolute-ref2.elem2
    dict-ref:
        $: interesting.stuff
        elem2: 'not FOO 2'

interesting:
    stuff:
        elem1: 'FOO 1'
        elem2: 'FOO 2'

interesting.word: 'cool'

foo: 'foo value'


 === PARSED CFG ===
references:
  absolute-ref1: foo value
  absolute-ref2:
    elem1: FOO 1
    elem2: FOO 2
  relative-ref1: foo value
  relative-ref2: FOO 2
  dict-ref:
    elem2: not FOO 2
    elem1: FOO 1
interesting:
  stuff:
    elem1: FOO 1
    elem2: FOO 2
  word: cool
foo: foo value



A referenced object can contain other reference. But cyclical definition is prohibited.

In [4]:
try:
    Cfg.Parser.resolve_refs(Cfg.Dict({'foo': '$foo2', 'foo2': '$foo'}))
except Cfg.Parser.Error as e:
    print('ParseError raised:', e)
else:
    raise RuntimeError('Recursive defintion detection failed!')

ParseError raised: Redundent definition of "foo2".


### Inheritage
Inherit another configuration file by starting the yaml with:
```yaml
%INHERIT file.yaml
---
```
Both file are merged into a single configuration. If the two files define the same field, the new value takes precedence over the inherited one.

In [5]:
cfg = Cfg.Parser('yaml-demos/inheritage.yaml').get_config(parse_obj=False)

with open('yaml-demos/inheritage.yaml', 'r') as f:
    raw = f.read()
print(' === RAW FILE ===')
print(raw)
print('\n === PARSED CFG ===')
print(cfg.to_yaml())

assert cfg['references.inherited-ref'] == 'cool'

 === RAW FILE ===
%INHERIT references.yaml
---

references:
    inherited-ref: $interesting.word

interesting:
    stuff:
        elem2: "NEW FOO"


 === PARSED CFG ===
references:
  absolute-ref1: foo value
  absolute-ref2:
    elem1: FOO 1
    elem2: NEW FOO
  relative-ref1: foo value
  relative-ref2: NEW FOO
  dict-ref:
    elem2: not FOO 2
    elem1: FOO 1
  inherited-ref: cool
interesting:
  stuff:
    elem1: FOO 1
    elem2: NEW FOO
  word: cool
foo: foo value



### Version Multiplicity
Several version of a configuration can be described in a single yaml file either:

-  by adding ``@`` at the end of any field name and defining it as the list of its value:
```yaml
field@: [value1, value2]
```

- by including several documents in the configuration file. The first document will be the base, each subsequent document will create new a version of the configuration: merging its values with the base.

Hence:
```yaml
base-stuff:
    foo: 'FOO'
specified-field: default_value
---
version: '1'
---
version: '2'
specified-field: other-value
```

Will produce two configurations:
```yaml
base-stuff:
    foo: 'FOO'
version: '1'
specified-field: default_value
```
and 
```yaml
base-stuff:
    foo: 'FOO'
version: '2'
specified-field: other-value
```

These syntaxes can be combined. In this case, each possible combinations of versions will be created.

In [6]:
cfgs = Cfg.Parser('yaml-demos/versions.yaml')

with open('yaml-demos/versions.yaml', 'r') as f:
    raw = f.read()
print(' === RAW FILE ===')
print(raw)
for i, cfg in enumerate(cfgs):
    print(f'\n === VERSION {i+1} ===')
    print(cfg.to_yaml())


assert cfgs[0]['document'] == 1
assert cfgs[0]['list'] == 'ele1'
assert cfgs[1]['document'] == 2
assert cfgs[2]['list'] == {'ele2': 'is a dictionnary'}

 === RAW FILE ===
list@: [ele1, {ele2: 'is a dictionnary'}]
---
document: 1
doc1-attr: foo
---
document: 2
doc2-attr: foo


 === VERSION 1 ===
list: ele1
document: 1
doc1-attr: foo


 === VERSION 2 ===
list: ele1
document: 2
doc2-attr: foo


 === VERSION 3 ===
list:
  ele2: is a dictionnary
document: 1
doc1-attr: foo


 === VERSION 4 ===
list:
  ele2: is a dictionnary
document: 2
doc2-attr: foo



cfgs = Cfg.Parser('yaml-demos/inherit-ref.yaml')

with open(cfgs.path, 'r') as f:
    raw = f.read()
print(' === RAW FILE ===')
print(raw)
for i, cfg in enumerate(cfgs):
    print(f'\n === VERSION {i+1} ===')
    print(cfg.to_yaml())
